<a href="https://colab.research.google.com/github/IA2021UR/practica-9-joferne/blob/main/practica9_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 9 Parte 1: Word embeddings

En este primer notebook veremos distintos word embeddings. Para ello usaremos una librería de procesamiento de lenguaje natural llamada [gensim](https://radimrehurek.com/gensim/index.html). 

## Modelo antiguo

Antes de hablar de los modelos modernos, conviene conocer los modelos antiguos y sus limitaciones. 

Una de las técnicas más utilizadas hasta hace poco para codificar frases era conocida como [bolsa de palabras (o *bag of words*)](https://en.wikipedia.org/wiki/Bag-of-words_model). Esta técnica transforma cada documento a un vector de enteros de longitud fija. 

Por ejemplo, para las siguientes dos frases:

1. A Juan le gusta ver películas. A María también le gusta.
2. A Juan le gusta ver partidos de fútbol. María odia el futbol. 

El modelo produce los siguientes vectores (cada fila de la siguiente tabla representa una de las frases).


| Palabras | A | Juan | le | gusta | ver | películas | María | también | partidos | fútbol | odia | el |
|---------|--------- | --------- | --------- | --------- | --------- | --------- | --------- | --------- | --------- | --------- | --------- | --------- |
|Frase 1|2 | 1 | 2 | 1 | 1 | 1 | 1 |1 | 0 | 0 | 0 | 0 |
|Frase 2|1 | 1 | 1 | 1 | 1 | 0 | 1 |0 | 1 | 1 | 1 | 1 |

Cada vector tiene 12 elementos, donde cada elemento cuenta el número de veces que una determinada palabra ocurre en dicho documento. 

Este modelo era muy efectivo, pero tenía varias limitaciones. En primer lugar se pierde información sobre el orden de las palabras. Por ejemplo, las frases "A María le gusta Juan" y "A Juan le gusta María" tienen vectores idénticos. La solución consiste en usar [n-gramas](https://en.wikipedia.org/wiki/N-gram). Además este modelo tiene la limitación de ser *sparse* (muchos ceros para cada vector) y tener una dimensionalidad muy alta. 

Otro de los problemas más importantes es que este modelo no aprende el significado de las palabras subyacentes, y como consecuencia la distancia entre vectores no refleja la similitud o diferencia en significado. Estos problemas se han resuelto con los word-embeddings.





## Word2Vec

El modelo [Word2Vec](https://arxiv.org/pdf/1301.3781.pdf) fue introducido en 2013 por investigadores de Google, y es capaz de incrustar las palabras en un vector de dimensión bja usando una red neuronal. El resultado es un modelo donde los vectores que están cerca en el espacio tienen significados similares basados en el contexto. Es decir, no existe un único modelo word2vec, sino que es un modelo que se entrena a partir de un dataset (dicho dataset puede ser la wikipedia, twitter, ...).

Vamos a ver que es posible hacer con uno de estos modelos. Para ello vamos a usar un modelo entrenado en el dataset de noticias de Google. Por el momento vamos a usar modelos entrenados en inglés, más adelante en la práctica veremos cómo usar modelos en español. 

Comenzamos descargando el dataset (esto puede costar un tiempo). 

In [1]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

[================================------------------] 64.5% 1073.1/1662.8MB downloaded

ConnectionResetError: ignored

Una tarea común es obtener las primeras palabras del vocabulario usado para construir el modelo. 

In [ ]:
for i, word in enumerate(wv.vocab):
    if i == 10:
        break
    print(word)

Ahora podemos ver cuál es la representación de una palabra. 

In [ ]:
wv['king']

Desafortunadamente el modelo no es capaz de inferir el vector asociado a palabras "raras". 

In [ ]:
try:
    vec_cameroon = wv['cameroon']
except KeyError:
    print("The word 'cameroon' does not appear in this model")

Estos modelos proporcionan distintas funciones para tratar varios problemas. 

Por ejemplo, podemos ver la similaridad entre palabras. 


In [ ]:
pairs = [
    ('car', 'minivan'),   # a minivan is a kind of car
    ('car', 'bicycle'),   # still a wheeled vehicle
    ('car', 'airplane'),  # ok, no wheels, but still a vehicle
    ('car', 'cereal'),    # ... and so on
    ('car', 'communism'),
]
for w1, w2 in pairs:
    print('%r\t%r\t%.2f' % (w1, w2, wv.similarity(w1, w2)))

Podemos también mostrar las 5 palabras más similares a *car*.

In [ ]:
print(wv.most_similar(positive=['car'], topn=5))

**Ejercicio**: Busca las 5 palabras más similares para Spain. 

In [ ]:
print(wv.most_similar(positive=['Spain'], topn=5))

Es posible también buscar analogías. Por ejemplo, para resolver la analogía *man is to king, as woman is to ...*, se debe ejecutar la siguiente instrucción.

In [ ]:
wv.most_similar_cosmul(positive=['king','woman'],negative=['man'])

**Ejercicio**: Da respuesta a las siguientes analogías. 

*Eat is to ate, as go is to ...*

In [ ]:
wv.most_similar_cosmul(positive=['ate','go'],negative=['eat'])

*Madrid is to Spain, as Berlin is to ...*

In [ ]:
wv.most_similar_cosmul(positive=['Spain','Berlin'],negative=['Madrid'])

También es posible encontrar palabras extrañas dentro de un grupo de palabras. 

Por ejemplo, ¿cuál de las siguientes palabras no encaja en la lista ``[Jupyter, Earth, Saturday, Mars, Moon]``?

In [ ]:
wv.doesnt_match(['Jupyter','Earth','Saturday','Mars','Moon'])

**Ejercicio** ¿Qué palabra no encaja en la siguiente lista ``[April, May, September, Monday, July]``?

In [ ]:
wv.doesnt_match(['April', 'May', 'September', 'Monday', 'July'])

## Glove y Fasttext

Además de word2vec, han aparecido otros embeddings. Los más conocidos son [Glove](https://nlp.stanford.edu/projects/glove/) y [FastText](https://fasttext.cc/). 

Como hemos explicado anteriormente, el método word2vec aprende la representación de las palabras mediante una red neuronal. En cambio estos otros dos embeddings funcionan de manera un poco distinta.

Glove se basa en técnicas de factorización de matrices. Para ello comienza construyendo una gran matriz con tantas filas y columnas como palabras. En esta matriz la entrada i,j indica el número de veces que la palabra i aparece en la misma frase que la palabra j. Seguidamente dicha matriz de co-ocurrencias se factoriza para producir una representación de baja dimensión.  

Tanto Glove como wor2vec tienen el problema de que no sirven para codificar palabras "raras" o que no aparecen en el vocabulario. Para resolver dicho problema surgió FastText. 

FastText es una extensión del modelo word2vec. En lugar de aprender vectores para cada palabra directamente, FastText representa cada palabra como un n-grama de caracteres. Por ejemplo, si tomamos la palabra artificial y usamos n=3, la representación de dicha palabra viene dada por <ar, art, rti, tif, ifi, fic, ici, ial, al> donde < y > indican respectivamente el principio y final de una palabra. 

Este método ayuda a capturar el signficado de palabras más cortas y permite comprender los sufijos y prefijos. Una vez que las palabras son partidas en n-gramas se entrena un modelo similar al de word2vec. Una ventaja de FastText es que funciona con palabras raras que no habían sido vistas anteriormente (cosa que no ocurría con los otros modelos). 



Desde el punto de vista de su uso no hay diferencias con respecto a word2vec. Vamos a comenzar descargando un modelo glove y otro modelo fasttext. 

In [ ]:
import gensim.downloader as api
wvGlove = api.load('glove-twitter-25')
wvFastText = api.load('fasttext-wiki-news-subwords-300')

**Ejercicio** ¿Qué otros modelos de fasttext y glove proporciona la librería gensym? Consulta lo que hace la función ``api.info()``.

In [ ]:
api.info()

**Ejercicio** Compara los resultados proporcionados por cada uno de los embeddings para los ejercicios presentados en el apartado anterior. Añade tantas celdas como necesites. 

In [ ]:
for i, word in enumerate(wvGlove.vocab):
    if i == 10:
        break
    print(word)

In [ ]:
for i, word in enumerate(wvFastText.vocab):
    if i == 10:
        break
    print(word)

In [ ]:
wvGlove['king']

In [ ]:
wvFastText['king']

In [ ]:
try:
    vec_cameroon = wvGlove['cameroon']
except KeyError:
    print("The word 'cameroon' does not appear in this model")

In [ ]:
try:
    vec_cameroon = wvFastText['cameroon']
except KeyError:
    print("The word 'cameroon' does not appear in this model")

In [ ]:
pairs = [
    ('car', 'minivan'),   # a minivan is a kind of car
    ('car', 'bicycle'),   # still a wheeled vehicle
    ('car', 'airplane'),  # ok, no wheels, but still a vehicle
    ('car', 'cereal'),    # ... and so on
    ('car', 'communism'),
]
for w1, w2 in pairs:
    print('%r\t%r\t%.2f' % (w1, w2, wvGlove.similarity(w1, w2)))

In [ ]:
pairs = [
    ('car', 'minivan'),   # a minivan is a kind of car
    ('car', 'bicycle'),   # still a wheeled vehicle
    ('car', 'airplane'),  # ok, no wheels, but still a vehicle
    ('car', 'cereal'),    # ... and so on
    ('car', 'communism'),
]
for w1, w2 in pairs:
    print('%r\t%r\t%.2f' % (w1, w2, wvFastText.similarity(w1, w2)))

In [ ]:
print(wvGlove.most_similar(positive=['car'], topn=5))

In [ ]:
print(wvFastText.most_similar(positive=['car'], topn=5))

In [ ]:
print(wvGlove.most_similar(positive=['spain'], topn=5))

In [ ]:
print(wvFastText.most_similar(positive=['Spain'], topn=5))

In [ ]:
wvGlove.most_similar_cosmul(positive=['king','woman'],negative=['man'])

In [ ]:
wvFastText.most_similar_cosmul(positive=['king','woman'],negative=['man'])

In [ ]:
wvGlove.most_similar_cosmul(positive=['ate','go'],negative=['eat'])

In [ ]:
wvFastText.most_similar_cosmul(positive=['ate','go'],negative=['eat'])

In [ ]:
wvGlove.most_similar_cosmul(positive=['spain','berlin'],negative=['madrid'])

In [ ]:
wvFastText.most_similar_cosmul(positive=['Spain','Berlin'],negative=['Madrid'])

In [ ]:
wvGlove.doesnt_match(['jupyter','earth','saturday','mars','moon'])

In [ ]:
wvFastText.doesnt_match(['Jupyter','Earth','Saturday','Mars','Moon'])

In [ ]:
wvGlove.doesnt_match(['april', 'may', 'september', 'monday', 'july'])

In [ ]:
wvFastText.doesnt_match(['April', 'May', 'September', 'Monday', 'July'])

## Otros idiomas

Los embeddings son particulares de cada idioma por lo que la representación de palabras en inglés no nos sirve cuando estamos trabajando en español. Afortunadamente, existen [modelos preentrenados para nuestro idioma](https://github.com/dccuchile/spanish-word-embeddings).

Su uso es relativamente sencillo. Debemos descargar el fichero de los pesos y cargarlo. Seguidamente podremos usarlo como hemos visto anteriormente. Por ejemplo vamos a cargar un modelo FastText. Comenzamos descargando el modelo.

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.es.vec -O wiki.es.vec

A continuación mostramos las primeras filas del modelo, como se puede ver el documento que acabamos de descargar contiene en cada línea un n-grama (recordar qué es lo que hace FastText) y a continuación su representación en forma de vector. 

In [ ]:
!head wiki.es.vec

La carga del modelo es un poco distinta a lo visto anteriormente. En concreto debemos usar el siguiente código (notar que no cargamos todo el documento sino solo los 100000 primeros n-gramas). 

In [ ]:
from gensim.models.keyedvectors import KeyedVectors
wordvectors_file_vec = 'wiki.es.vec'
cantidad = 100000
wvFastTextSpanish = KeyedVectors.load_word2vec_format(wordvectors_file_vec, limit=cantidad)

Ahora podemos utilizar las mismas funciones vistas anteriormente. 

**Ejercicio:** Encuentra las 5 palabras más similares a bicicleta. 

In [ ]:
print(wvFastTextSpanish.most_similar(positive=['bicicleta'], topn=5))

**Ejercicio:** Responde a la siguiente analogía. *Hombre es a actor como mujer es a...*

In [ ]:
wvFastTextSpanish.most_similar_cosmul(positive=['actor','mujer'],negative=['hombre'])

**Ejercicio:** Responde a la siguiente analogía. *Canta es a cantar como juega es a ...*

In [ ]:
wvFastTextSpanish.most_similar_cosmul(positive=['cantar','juega'],negative=['canta'])

**Ejercicio:** Responde a la siguiente analogía. *Madrid es a España como Lisboa es a ...*

In [ ]:
wvFastTextSpanish.most_similar_cosmul(positive=['españa','lisboa'],negative=['madrid'])

**Ejercicio:** Encuentra la palabra que no encaja en la lista ``[lunes, martes, septiembre, jueves, viernes]``

In [ ]:
wvFastTextSpanish.doesnt_match(['lunes', 'martes', 'septiembre', 'jueves', 'viernes'])

**Ejercicio:** Descarga alguno de los otros modelos proporcionados en el [zoo de modelos](https://github.com/dccuchile/spanish-word-embeddings) y encuentra un ejemplo de analogía y otro de palabra que no encaje usando dicho modelo (añade tantas celdas como necesites).

In [2]:
!wget https://zenodo.org/record/3234051/files/embeddings-l-model.vec?download=1 -O suc.vec

--2021-05-12 10:09:55--  https://zenodo.org/record/3234051/files/embeddings-l-model.vec?download=1
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3446609027 (3.2G) [application/octet-stream]
Saving to: ‘suc.vec’

suc.vec             100%[===================>]   3.21G  6.73MB/s    in 5m 41s  

2021-05-12 10:15:38 (9.63 MB/s) - ‘suc.vec’ saved [3446609027/3446609027]



In [3]:
from gensim.models.keyedvectors import KeyedVectors
wordvectors_file_vec = 'suc.vec'
cantidad = 100000
wvFastTextSUC = KeyedVectors.load_word2vec_format(wordvectors_file_vec, limit=cantidad)

In [4]:
wvFastTextSUC.most_similar_cosmul(positive=['actor','mujer'],negative=['hombre'])

[('actriz', 1.029092788696289),
 ('dramaturga', 0.9112269878387451),
 ('coreógrafa', 0.9106628894805908),
 ('locutora', 0.88897305727005),
 ('compositora', 0.8822876214981079),
 ('escritora', 0.8775303959846497),
 ('actrices', 0.8739449977874756),
 ('pintora', 0.8698908090591431),
 ('presentadora', 0.868553876876831),
 ('actoral', 0.8673722743988037)]

In [5]:
wvFastTextSUC.doesnt_match(['lunes', 'martes', 'septiembre', 'jueves', 'viernes'])

/usr/local/lib/python3.7/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'septiembre'

Recuerda guardar tu notebook en GitHub con la opción *Save in GitHub* del menú *File*.